In [ ]:
import os

import pandas as pd

""" Loads data saved in /rsa

"""

In [2]:
%load_ext autoreload
%autoreload 2

from neuralmonkey.classes.session import load_mult_session_helper
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

#%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### MULT RES PLOTS

In [3]:
import pickle
import seaborn as sns
import pandas as pd
from pythonlib.tools.pandastools import grouping_print_n_samples, append_col_with_grp_index
from pythonlib.tools.plottools import rotate_x_labels, rotate_y_labels, savefig
from pythonlib.tools.snstools import rotateLabel
from pythonlib.tools.expttools import writeDictToYaml


In [204]:
list_which_level = ["stroke", "stroke_off"]
# version_distance = "pearson"

def load_mult_data_helper(animal, DATE, version_distance):
    """ Load all data across (i) timw widopws (ii) which levels
    """
    SAVEDIR = "/gorilla1/analyses/recordings/main/RSA"
    SAVEDIR_MULT = f"{SAVEDIR}/{animal}/MULT/{DATE}/{version_distance}"
    import os
    os.makedirs(SAVEDIR_MULT, exist_ok=True)
    
    params = {
        "DATE":DATE,
        "animal":animal,
        "list_which_level":list_which_level,
        "version_distance":version_distance,
        "SAVEDIR":SAVEDIR
    }
    path = f"{SAVEDIR_MULT}/params.yaml"
    writeDictToYaml(params, path)
    
    ####### LOAD    
    RES = []
    for which_level in list_which_level:
        print("Getting: ", which_level)
        savedir = f"{SAVEDIR}/{animal}/{DATE}/{which_level}/{version_distance}"
        path = f"{savedir}/resthis.pkl"
        print("Loading res from: ", path)
        try:
            with open(path, "rb") as f:
                res = pickle.load(f)
            RES.append(res)
        except Exception as err:
            print(path)
            print("Couldnt load this data! *******************", version_distance, animal, DATE, which_level)
        RES.append(res)
        
    # bregions
    from neuralmonkey.neuralplots.brainschematic import REGIONS_IN_ORDER
    
    return RES, SAVEDIR_MULT, params, REGIONS_IN_ORDER

In [5]:
# Either run above and create RES, or load here
# DATE = 230623
# animal = "Pancho"

for animal, DATE in [
    ("Diego", "230628"),
    ("Diego", "230630"),
    ("Pancho", "230623"),
    ("Pancho", "230626"),
    ]:
    for version_distance in ["pearson", "euclidian"]:
    
        # DATE = 230630
        # animal = "Diego"
    
        RES, SAVEDIR_MULT, params, REGIONS_IN_ORDER = load_mult_data_helper(animal, DATE, version_distance)

        # COLLEC 
        # Compare to theoetical simmat
        list_df = []
        for res in RES:
            which_level = res["which_level"] 
            version_distance = res["version_distance"] 
            df = res["DFRES_THEOR"]
            df["which_level"] = which_level
            df["version_distance"] = version_distance
            list_df.append(df)
            
        DFMULT_THEOR = pd.concat(list_df).reset_index(drop=True)
        
        DFMULT_THEOR
        
        # a = DFMULT_THEOR["bregion"]=="vlPFC_p"
        # b = DFMULT_THEOR["var"]=="shape_oriented"
        # c = DFMULT_THEOR["which_level"]=="stroke"
        # d = DFMULT_THEOR["twind"]==(-0.1, 0.1)
        # 
        # DFMULT_THEOR[(a) & (b) & (c) & (d)]
        
        # grouping_print_n_samples(DFMULT_THEOR, ["bregion", "var", "which_level", "twind"])
        
        ##### Preprocess
        DFMULT_THEOR["twind_str"] = ["_to_".join([str(tt) for tt in t]) for t in DFMULT_THEOR["twind"].tolist()]
        DFMULT_THEOR = append_col_with_grp_index(DFMULT_THEOR, ["which_level", "twind_str"], "wl_tw", strings_compact=True)
        EFFECT_VARS = DFMULT_THEOR["var"].unique().tolist()
        list_bregion = DFMULT_THEOR["bregion"].unique().tolist()
        
        ##### Plots
        savedir = f"{SAVEDIR_MULT}/overview"
        os.makedirs(savedir, exist_ok=True)
        from pythonlib.tools.pandastools import append_col_with_grp_index, convert_to_2d_dataframe
        
        for yvar in ["cc", "mr_coeff"]:
            
            # (1) Pointplot, showing all results, but hard to read.
            fig = sns.catplot(data=DFMULT_THEOR, x="twind", y=yvar, col="bregion", hue="var", kind="point", row="which_level")
            rotateLabel(fig)
            savefig(fig, f"{savedir}/pointplot-{yvar}-bregions.pdf")
            plt.close("all")
        
            # IN PROGRESS - subtracting global mean within each level of (effect var). Decided it wasnt needed
            # conjucntion of twind and which_level
            if False:
                from pythonlib.tools.pandastools import datamod_normalize_row_after_grouping
                # datamod_normalize_row_after_grouping(DFMULT_THEOR, "bregion", 
                grp = ["bregion"]
                DFMULT_THEOR.groupby(grp).transform(lambda x: (x - x.mean()) / x.std())
                
                # Normalize by subtracting mean effect within each bregion, to allow comparison of each bregion's "signature"
                from pythonlib.tools.pandastools import aggregGeneral
                aggregGeneral(DFMULT_THEOR, ["var" , ""])
            
            if False:
                # IN PROGRESS - ONE VECTOR FOR EACH BREGION (across var, which_level, and time window).
                # new variable, conjunction of var and time window
                %matplotlib inline
                from pythonlib.tools.pandastools import append_col_with_grp_index, convert_to_2d_dataframe
                DFMULT_THEOR = append_col_with_grp_index(DFMULT_THEOR, ["var", "wl_tw"], "var_wl_tw", strings_compact=True)
                
                # Heatmap
                ncols = 3
                W = 4
                H = 4
                nrows = int(np.ceil(len(EFFECT_VARS)/ncols))
                dfthis = DFMULT_THEOR
                for norm_method in [None, "row_sub", "col_sub"]:
                    _, fig, _, _ = convert_to_2d_dataframe(dfthis, "bregion", "var_wl_tw", True, "mean", yvar, annotate_heatmap=False, dosort_colnames=False,
                                            norm_method=norm_method)
                    savefig(fig, f"{savedir}/heatmap-")
                    
            # (2) Heatmaps, easier to parse (Concatting all which_levels)
            W = 4
            H = 4
            ncols = 3
            for norm_method in [None, "all_sub", "row_sub", "col_sub"]:
                # Heatmap - one subplot for each var
                nrows = int(np.ceil(len(EFFECT_VARS)/ncols))
                fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
                for var, ax in zip(EFFECT_VARS, axes.flatten()):
                    print(var)
                    dfthis = DFMULT_THEOR[DFMULT_THEOR["var"]==var]
                    convert_to_2d_dataframe(dfthis, "bregion", "wl_tw", True, "mean", yvar, annotate_heatmap=False, dosort_colnames=False,
                                            norm_method=norm_method, ax=ax)
                    ax.set_title(var)
                savefig(fig, f"{savedir}/heatmap-subplot_by_var-norm_{norm_method}-ccvar_{yvar}.pdf")
        
            
                # Heatmap - one subplot for each bregion
                nrows = int(np.ceil(len(list_bregion)/ncols))
                fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
                for bregion, ax in zip(list_bregion, axes.flatten()):
                    print(norm_method, bregion)
                    dfthis = DFMULT_THEOR[DFMULT_THEOR["bregion"]==bregion]
                    convert_to_2d_dataframe(dfthis, "var", "wl_tw", True, "mean", yvar, annotate_heatmap=False, dosort_colnames=False,
                                            norm_method=norm_method, ax=ax)
                    ax.set_title(bregion)
                savefig(fig, f"{savedir}/heatmap_concat-subplot_by_bregion-norm_{norm_method}-ccvar_{yvar}.pdf")
                
                plt.close("all")
                
            # (3) Heatmaps, easier to parse (Separate plots for each which_levels)
            W = 4
            H = 4
            ncols = 3
            for which_level in list_which_level:
                for norm_method in [None, "all_sub", "row_sub", "col_sub"]:
                    # Heatmap - one subplot for each var
                    nrows = int(np.ceil(len(EFFECT_VARS)/ncols))
                    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
                    for var, ax in zip(EFFECT_VARS, axes.flatten()):
                        print(var)
                        dfthis = DFMULT_THEOR[(DFMULT_THEOR["var"]==var) & (DFMULT_THEOR["which_level"]==which_level)]
                        convert_to_2d_dataframe(dfthis, "bregion", "twind", True, "mean", yvar, annotate_heatmap=False, dosort_colnames=False,
                                                norm_method=norm_method, ax=ax)
                        ax.set_title(var)
                    savefig(fig, f"{savedir}/heatmap_whichlevel_{which_level}-subplot_by_var-norm_{norm_method}-ccvar_{yvar}.pdf")
            
                
                    # Heatmap - one subplot for each bregion
                    nrows = int(np.ceil(len(list_bregion)/ncols))
                    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
                    for bregion, ax in zip(list_bregion, axes.flatten()):
                        print(norm_method, bregion)
                        dfthis = DFMULT_THEOR[(DFMULT_THEOR["bregion"]==bregion) & (DFMULT_THEOR["which_level"]==which_level)]
                        convert_to_2d_dataframe(dfthis, "var", "twind", True, "mean", yvar, annotate_heatmap=False, dosort_colnames=False,
                                                norm_method=norm_method, ax=ax)
                        ax.set_title(bregion)
                    savefig(fig, f"{savedir}/heatmap_whichlevel_{which_level}-subplot_by_bregion-norm_{norm_method}-ccvar_{yvar}.pdf")
                    
                    plt.close("all")
                    
        ##### Kernels - each bregions "signature" by concatting across all times and var
        savedir = f"{SAVEDIR_MULT}/kernels"
        os.makedirs(savedir, exist_ok=True)
        
        def kernel_convert_from_combined_to_split(kernel_combined):
            """ Convert from list of tuple (event string, weight) to
            two lists; events and weights.
            """
            kernel = [k[0] for k in kernel_combined]
            weights = [k[1] for k in kernel_combined]
            
            return kernel, weights
            
        def kernel_extract_which_var(kernel_name):
            
            map_name_to_var = {
                "reach_prev":"gap_from_prev_angle_binned",
                "reach_next":"gap_to_next_angle_binned",
                "stkidx_stroke":"stroke_index_semantic",
                "shape":"shape_oriented",
                "loc_future":"CTXT_loc_next",
                "shape_future":"CTXT_shape_next",
                "stkidx_entire":"stroke_index_semantic"}
            
            if kernel_name not in map_name_to_var.keys():
                # The name is the var
                return kernel_name
            else:
                return map_name_to_var[kernel_name]
            
        def _check_twind_center_within(twind, tmin, tmax):
            """ Return True if the center of the window (twind) 
            is greater than tmin and les sthan tmax"""
            cen = np.mean(twind)
            return cen>tmin and cen<tmax
        
        # Construct kernels for each (i.e., events in order and weights in order)
        list_twind = DFMULT_THEOR["twind"].unique().tolist()
        list_which_level = DFMULT_THEOR["which_level"].unique().tolist()
        
        kernel_reach_prev = [] 
        kernel_reach_next = [] 
        kernel_gridloc = [] 
        kernel_shape = [] 
        kernel_future_loc = []
        kernel_future_shape = []
        kernel_stkidx_during = []
        kernel_stkidx_entire = []
        for wl in list_which_level:
            for twind in list_twind:
                ev = f"{wl}|{twind[0]}_to_{twind[1]}"
        
                # Reach motor
                if wl=="stroke" and _check_twind_center_within(twind, -0.5, -0.1):
                    # then window ends before align time
                    k = 1
                else:
                    k = 0
                kernel_reach_prev.append((ev, k))
                    
                # After offset
                if wl=="stroke_off" and _check_twind_center_within(twind, 0, 0.5):
                    # then window ends before align time
                    k = 1
                else:
                    k = 0
                kernel_reach_next.append((ev, k))
                
                # Location (abstract, current stroke)
                if wl=="stroke" and _check_twind_center_within(twind, -0.1, 0.3):
                    # then window ends before align time
                    k = 1        
                # elif wl=="stroke_off" and _check_twind_center_within(twind, -0.3, 0):
                #     k = 1
                else:
                    k = 0
                kernel_gridloc.append((ev, k))
                kernel_stkidx_during.append((ev, k))
                kernel_shape.append((ev, k))
                
                # Next stroke (loc)
                if wl=="stroke" and _check_twind_center_within(twind, -0.5, 0.3):
                    # then window ends before align time
                    k = 1        
                else:
                    k = 0
                kernel_future_loc.append((ev, k))
                kernel_future_shape.append((ev, k))
                
                # Stroke index (entire)
                k=1
                kernel_stkidx_entire.append((ev, k))
        
        KERNELS = {
            "reach_prev":kernel_reach_prev,
            "reach_next":kernel_reach_next,
            "gridloc":kernel_gridloc,
            "stkidx_stroke":kernel_stkidx_during,
            "shape":kernel_shape,
            "loc_future":kernel_future_loc,
            "shape_future":kernel_future_shape,
            "stkidx_entire":kernel_stkidx_entire,
        }
        
        # Plot kernel templates
        fig, ax = plt.subplots()
        
        for i, (name, kernel) in enumerate(KERNELS.items()):
            k, w= kernel_convert_from_combined_to_split(kernel)
            
            var = kernel_extract_which_var(name)
            
            ax.scatter(k, np.ones(len(k))*i, c=[1-x for x in w], alpha=0.5, label=name)
            ax.text(0, i, var, color="r", alpha=0.5)
            
        ax.set_yticks(list(range(len(KERNELS.keys()))), labels=KERNELS.keys())
        ax.set_ylabel("kernel name")
        ax.set_xlabel("time window")
        rotate_x_labels(ax, 90)
        rotate_y_labels(ax, 0)
        ax.set_title("Kernels (dark dot = 1); red: var it operates on")            
        savefig(fig, f"{savedir}/kernels_weights.pdf")
        
        
        yvar = "cc"
        for yvar in ["cc", "mr_coeff"]:
            # PREPROCESS - # for each variable, get 2d df (bregion x twinds)
            from neuralmonkey.analyses.event_temporal_modulation import kernel_compute, _kernel_compute_scores
            DictBregionToDf2d = {}
            for bregion in list_bregion:
                dfthis = DFMULT_THEOR[DFMULT_THEOR["bregion"]==bregion]
                dftmp, fig, ax, rgba_values = convert_to_2d_dataframe(dfthis, "var", "wl_tw", False, "mean", yvar, annotate_heatmap=False, dosort_colnames=False)
                DictBregionToDf2d[bregion] = dftmp
            DictVarToDf2d = {}
            for var in EFFECT_VARS:
                dfthis = DFMULT_THEOR[DFMULT_THEOR["var"]==var]
                dftmp, fig, ax, rgba_values = convert_to_2d_dataframe(dfthis, "bregion", "wl_tw", False, "mean", yvar, annotate_heatmap=False, dosort_colnames=False)
                DictVarToDf2d[var] = dftmp
            
            ##### Score data using kernesl
            res = []
                
            for i, (name, kernel) in enumerate(KERNELS.items()):
                k, w= kernel_convert_from_combined_to_split(kernel)
                var = kernel_extract_which_var(name)
                dfthis = DictVarToDf2d[var]
                
                # apply kernel
                scores = _kernel_compute_scores(dfthis, k, w)
                
                # Distribute scores across bregions
                assert list_bregion==dfthis.index.tolist()
                
                # Collect
                for s, br in zip(scores, list_bregion):
                    
                    res.append({
                        "bregion":br,
                        "score":s,
                        "kernel_name":name,
                        "kernel_events":k,
                        "kernel_weights":w,
                        "var":var
                    })
                
            dfres_kernels = pd.DataFrame(res)  
            dfres_kernels
            
            # Second-order kernels, whose dimensions are the first-order kernels
            # NOTE: First-order kernels operate over time windows (for a specific var)
            # --> Therefore, second-order kernels are 2d-kernels, operating first over time, then over variables.
            # NOTE: many first order kernels are perfectly fine to keep as second-order..
            
            # first, get reshaped df (bregion, first order kernel name)
            # dfres_kernels_2d = convert_to_2d_dataframe(dfres_kernels, "bregion", "kernel_name", False, "mean", "score", dosort_colnames=False, list_cat_1=REGIONS_IN_ORDER)[0]
            dfres_kernels_2d = convert_to_2d_dataframe(dfres_kernels, "bregion", "kernel_name", False, "mean", "score", dosort_colnames=False)[0]
            
            k = ("reach_next", "reach_prev")
            w = (1,1)
            name = "reachdir"
            scores = _kernel_compute_scores(dfres_kernels_2d, k, w)
            dfres_kernels_2d[name] = scores
            
            k = ("gridloc", "reachdir")
            w = (1,-1)
            name = "gridloc_abstract"
            scores = _kernel_compute_scores(dfres_kernels_2d, k, w)
            dfres_kernels_2d[name] = scores
            
            
            from pythonlib.tools.snstools import heatmap
            fig, axes = plt.subplots(2,2, figsize=(10,10))
            for ax, norm_method in zip(axes.flatten(), [None, "all_sub", "col_sub", "row_sub"]):
                heatmap(dfres_kernels_2d, ax, False, (None, None), norm_method=norm_method)
                # convert_to_2d_dataframe(dfres_kernels, "bregion", "kernel_name", True, "mean", "score", annotate_heatmap=False, norm_method="col_sub", dosort_colnames=False, list_cat_1=REGIONS_IN_ORDER)
                ax.set_title(f"norm_{norm_method}")
            savefig(fig, f"{savedir}/heatmap-kernel_scores-ccvar_{yvar}.pdf")
            
            plt.close("all")

AssertionError: 

In [ ]:

# Pick out specific pairs to contrast (plot)
from pythonlib.tools.pandastools import plot_45scatter_means_flexible_grouping
plot_45scatter_means_flexible_grouping(dfres_kernels_2d, "kernel_name", "gridloc_abstract", "reachdir", None, "")

In [ ]:
# And then PCA.

# kernels, i.e., a mask of length (var x which_level x twind)



##### Testing out, method for directly pitting two variables against each other

In [ ]:
# 1) Prune at level of comparisons: keep only comparisons that are opposed across the vars.
# -- What is the principle behind this? IT is more about the sign of the effect than the magnitude?
# -- More generally, is about getting subset of comparisons which are not correlated across the two vars.
# -- This seems weird?
# 2) Prune at level of var-levels: keep only levels (of both vars) that have at least 1 case of other var.
# -- This would get the "same beh" control in epochs.
# 3) "Kernels" using either the raw results, or the processed ones here (pairwise between variables).

#

In [ ]:
#TODO:
# - pick specific pair of variables. and plot across (bregion, timewindows). 


In [ ]:
# Pull out speciifc dataset (bregion x time)
which_level = "stroke"
version_distance = "pearson"
PLOT = False

savedir = f"{SAVEDIR}/{animal}/{DATE}/{which_level}/{version_distance}"
path = f"{savedir}/resthis.pkl"
print("Loading res from: ", path)
try:
    with open(path, "rb") as f:
        res = pickle.load(f)
except Exception as err:
    print(path)
    print("Couldnt load this data! *******************", version_distance, animal, DATE, which_level)

## EXTRACT
DictBregionTwindClraw = res["DictBregionTwindClraw"]
DictBregionTwindClsim = res["DictBregionTwindClsim"]
DictBregionTwindCPA = res["DictBregionTwindPA"]

res = []
for twind in list_twind:
    for bregion in list_bregion:
        # bregion = "PMd_p"
        key = (bregion, twind)
        Clraw = DictBregionTwindClraw[key]
        Clsim = DictBregionTwindClsim[key]
        PA = DictBregionTwindCPA[key]
        
        ##### 1. Given two variables, how similar are their theoretical sim mats?
        if PLOT:
            list_var1 =[]
            list_var2 = []
            list_cc = []
            for i, var1 in enumerate(EFFECT_VARS):
                for j, var2 in enumerate(EFFECT_VARS):
                    if j>i:
                        Cltheor1, _ = Clsim.rsa_distmat_construct_theoretical(var1, False)
                        Cltheor2, _ = Clsim.rsa_distmat_construct_theoretical(var2, False)
            
                        vec1 = Cltheor1.dataextract_upper_triangular_flattened()
                        vec2 = Cltheor2.dataextract_upper_triangular_flattened()
                        c = np.corrcoef(vec1, vec2)[0,1]
                        
                        print(var1, " -- ", var2, " -- ", c)
                        list_var1.append(var1)
                        list_var2.append(var2)
                        list_cc.append(c)
            
            dfthis = pd.DataFrame({"var1":list_var1, "var2":list_var2, "cc":list_cc})
            convert_to_2d_dataframe(dfthis, "var1", "var2", True, "mean", "cc", annotate_heatmap=True, dosort_colnames=False)
    
        # from neuralmonkey.analyses.state_space_good import rsa_convert_PA_to_Cl
        # rsa_convert_PA_to_Cl(DictBregionTwindPA.values[0])
        
        ##### Compare pair of vars
        # Go thru all pairs of vars
        PLOT = False
        for ivar, var in enumerate(EFFECT_VARS):
            for jvar, var_other in enumerate(EFFECT_VARS):
                if not var==var_other:
                # if jvar>ivar:
                    # var = "gridloc"
                    # var_other = "gap_from_prev_angle_binned"
                    # var = "shape_oriented"
                    # var_other = "gridloc"
                    
                    # find pairs of datapts which make different predictions across the two variables.
                    Cltheor1, _ = Clsim.rsa_distmat_construct_theoretical(var, False)
                    Cltheor2, _ = Clsim.rsa_distmat_construct_theoretical(var_other, False)
                    
                    # Extract actual data and theor data
                    vec_dat = Clsim.dataextract_upper_triangular_flattened()
                    vec1 = Cltheor1.dataextract_upper_triangular_flattened()
                    vec2 = Cltheor2.dataextract_upper_triangular_flattened()
                    
                    cc1 = np.corrcoef(vec_dat, vec1)[0,1]
                    cc2 = np.corrcoef(vec_dat, vec2)[0,1]
                    print(var, cc1)
                    print(var_other, cc2)
                    
                    cc_kind = "actual"
                    res.append({
                        "var":var,
                        "var_other":var_other,
                        "cc1":cc1,
                        "cc2":cc2,
                        "cc_kind":cc_kind,
                        "bregion":bregion,
                        "twind":twind
                    })
                    # res.append({
                    #     "var_or_other":"var",
                    #     "var":var,
                    #     "cc":cc1,
                    #     "cc_kind":cc_kind,
                    #     "bregion":bregion,
                    #     "twind":twind
                    # })
                    # res.append({
                    #     "var_or_other":"other",
                    #     "var":var_other,
                    #     "cc":cc2,
                    #     "cc_kind":cc_kind,
                    #     "bregion":bregion,
                    #     "twind":twind
                    # })
                    
                    ##### Method 1: include only data-pairs that make different predictions across the var
                    
                    # - normalize so that 0 is same and 1 is most different. This is becuase some, e.g, for
                    # ordinal, have larger range.
                    vec1_norm = vec1/np.max(vec1)
                    vec2_norm = vec2/np.max(vec2)
                    
                    inds_mask = vec1_norm!=vec2_norm
                    print(f"Keeping this many comparisons: {sum(inds_mask)}, out of this many total: {len(inds_mask)}")
                    
                    cc1 = np.corrcoef(vec_dat[inds_mask], vec1[inds_mask])[0,1]
                    cc2 = np.corrcoef(vec_dat[inds_mask], vec2[inds_mask])[0,1]
                    print(var, cc1)
                    print(var_other, cc2)
                    
                    cc_kind = "method1"
                    res.append({
                        "var":var,
                        "var_other":var_other,
                        "cc1":cc1,
                        "cc2":cc2,
                        "cc_kind":cc_kind,
                        "bregion":bregion,
                        "twind":twind
                    })
                    
                    ##### Method 2: Constrain just to cases that are "same" (i.e., dist = 0) in the other var. 
                    # NO: this is not good. this removes all comparisons of var across levels of other_var. This stops 
                    # testing of "abstraction" of var.
                    if False:
                        inds_mask = vec2==1
                        print(f"Keeping this many comparisons: {sum(inds_mask)}, out of this many total: {len(inds_mask)}")
                        
                        cc1 = np.corrcoef(vec_dat[inds_mask], vec1[inds_mask])[0,1]
                        cc2 = np.corrcoef(vec_dat[inds_mask], vec2[inds_mask])[0,1]
                        print(cc1, cc2)
                    
                    ##### Method 3: Constrain just to levels of "othervar" that have at least 2 levels for var. 
                    # - reason: otherwise that level of othervar isn't contributing.
                    from pythonlib.tools.pandastools import conjunction_vars_prune_to_balance, extract_with_levels_of_conjunction_vars
                    
                    # First, get subset of rows that are good.
                    dflab = Clsim.rsa_labels_return_as_df()
                    if PLOT:
                        savedir="/tmp/tmp.png"
                    else:
                        savedir = None
                    dfout1, _ = extract_with_levels_of_conjunction_vars(dflab, var, [var_other], n_min=2, lenient_allow_data_if_has_n_levels=2, plot_counts_heatmap_savedir=savedir, DEBUG=False);
                    dfout2, _ = extract_with_levels_of_conjunction_vars(dflab, var_other, [var], n_min=2, lenient_allow_data_if_has_n_levels=2, plot_counts_heatmap_savedir=savedir, DEBUG=False);
                    inds_rows = [i for i in dfout1["row_index"].tolist() if i in dfout2["row_index"].tolist()]
                    
                    print(len(dflab), " datapts, pruned to --> ", len(inds_rows))
                    
                    vec_dat = Clsim.dataextract_upper_triangular_flattened(True, inds_rows)
                    vec1 = Cltheor1.dataextract_upper_triangular_flattened(True, inds_rows)
                    vec2 = Cltheor2.dataextract_upper_triangular_flattened(True, inds_rows)
                    
                    cc1 = np.corrcoef(vec_dat, vec1)[0,1]
                    cc2 = np.corrcoef(vec_dat, vec2)[0,1]
                    print(var, cc1)
                    print(var_other, cc2)
                    
                    cc_kind = "method3"
                    res.append({
                        "var":var,
                        "var_other":var_other,
                        "cc1":cc1,
                        "cc2":cc2,
                        "cc_kind":cc_kind,
                        "bregion":bregion,
                        "twind":twind
                    })
                    
                    if False:
                        # To get fully square matrix. Not worth it - drops too much data.
                        dfthisout, dfcounts = conjunction_vars_prune_to_balance(dflab, var, var_other, True, prefer_to_drop_which=var);
                    
                    if PLOT:    
                        # Plot correlation
                        fig, ax = plt.subplots()
                        # ax.plot(vec1[inds_mask], vec_dat[inds_mask], "xk", alpha=0.1)
                        ax.plot(vec2[inds_mask], vec_dat[inds_mask], "xk", alpha=0.1)
                        ax.set_xlabel("dist(theor)")
                        ax.set_ylabel("dist(dat)")
                        
                    plt.close("all")


In [ ]:
# Plot summary
dfres_cc_pairs = pd.DataFrame(res)

# Heatmap (subplot for each bregion, each showing each pair of variables)
cc_kind = "method1"

if cc_kind=="method1":
    diverge=True
else:
    diverge=False
ncols = 3
W = 4
H = 4
nrows = int(np.ceil(len(list_bregion)/ncols))

# for norm_method in ["all_sub", None, "row_sub", "col_sub"]:
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
for bregion, ax in zip(list_bregion, axes.flatten()):
    print(bregion)
    a = dfres_cc_pairs["bregion"]==bregion
    b = dfres_cc_pairs["cc_kind"]==cc_kind    
    dfthis = dfres_cc_pairs[(a) & (b)]
    convert_to_2d_dataframe(dfthis, "var", "var_other", True, "mean", "cc1", ax=ax, annotate_heatmap=False, diverge=diverge)
    ax.set_title(bregion) 


In [ ]:
# Heatmap (subplot for each bregion, each showing each pair of variables)
cc_kind = "method3"

if cc_kind=="method1":
    diverge=True
else:
    diverge=False
ncols = 3
W = 4
H = 4
nrows = int(np.ceil(len(list_bregion)/ncols))

# for norm_method in ["all_sub", None, "row_sub", "col_sub"]:
fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
for bregion, ax in zip(list_bregion, axes.flatten()):
    print(bregion)
    a = dfres_cc_pairs["bregion"]==bregion
    b = dfres_cc_pairs["cc_kind"]==cc_kind    
    dfthis = dfres_cc_pairs[(a) & (b)]
    convert_to_2d_dataframe(dfthis, "var", "var_other", True, "mean", "cc1", ax=ax, annotate_heatmap=False, diverge=diverge)
    ax.set_title(bregion) 


In [ ]:
# Process results from method 1: for each var take mean over all other vars.
from pythonlib.tools.pandastools import aggregGeneral
dfres_cc_pairs_AGG = aggregGeneral(dfres_cc_pairs, ["var", "cc_kind", "bregion", "twind"], ["cc1"], aggmethod=["mean"])
dfres_cc_pairs_AGG

In [ ]:
list_cc_method = dfres_cc_pairs_AGG["cc_kind"].unique().tolist()
cc_kind = "method3"

ncols = 3
W = 4
H = 4
nrows = int(np.ceil(len(list_cc_method)/ncols))


fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*W, nrows*H))
for cc_kind, ax in zip(list_cc_method, axes.flatten()):
    if cc_kind=="method1":
        diverge=True
    else:
        diverge=False
    print(cc_kind)
    # a = dfres_cc_pairs["bregion"]==bregion
    b = dfres_cc_pairs_AGG["cc_kind"]==cc_kind    
    dfthis = dfres_cc_pairs_AGG[(b)]
    convert_to_2d_dataframe(dfthis, "bregion", "var", True, "mean", "cc1", ax=ax, annotate_heatmap=False, diverge=diverge, dosort_colnames=False)
    ax.set_title(cc_kind) 


##### Summary plot across all data

### Post-hoc plotting things realtied to sim mat

In [10]:
from neuralmonkey.analyses.state_space_good import _preprocess_rsa_scalar_population

In [78]:
RES[0].keys()

In [81]:

version_distance = "pearson"
RES, SAVEDIR_MULT, params, list_bregion = load_mult_data_helper(animal, DATE, version_distance)

resthis =[]
resthis = []
for res in RES:
    for bregion in list_bregion:
        for twind in res["list_time_windows"]:
            key = (bregion, twind)
            PA = res["DictBregionTwindPA"][key]
            # Clraw = res["DictBregionTwindClraw"][key]
            # Clsim = res["DictBregionTwindClsim"][key]
            EFFECT_VARS = res["EFFECT_VARS"]
    
            # Norm of activity for each level of the variable. Then average these norms over the levels.    
            for var in EFFECT_VARS:
                # Extract raw data
                _, _, Clraw, _, PAagg = _preprocess_rsa_scalar_population(PA, [var], res["version_distance"], 
                                                  res["subtract_mean_each_level_of_var"],
                                                  PLOT = False, sdir=sdir, PLOT_THEORETICAL_SIMMATS=False,
                                                COMPUTE_SAME_DIFF_DIST = False, COMPUTE_VS_THEOR_MAT = False)
                
                norms = np.linalg.norm(Clraw.Xinput, axis=1)
                # Then take avereage
                np.mean(norms)
                
                # break it into levels of responses
                for normthis, lev in zip(norms, Clraw.Labels):
                    assert len(lev)==1, "should be 1-tuple"
                    resthis.append({
                        "bregion":bregion,
                        "twind":twind,
                        "which_level":res["which_level"],
                        "var":var,
                        "var_level":lev[0],
                        "norm_pop_vec":normthis
                    })    


In [ ]:
# correlation between these variables
# TODO: extract values from within _preprocess_rsa_scalar_population to do this.

# %matplotlib inline
# Cltheor = Clsim.rsa_distmat_construct_theoretical("shape_oriented")[0]
# Cltheor.rsa_plot_heatmap()

#### Compute effect size of a given variable

In [153]:
savedir = f"{SAVEDIR_MULT}/effects_each_var"
os.makedirs(savedir, exist_ok=True)
print(savedir)

/gorilla1/analyses/recordings/main/RSA/Pancho/MULT/230626/pearson/effects_each_var


In [192]:
Clraw.rsa_labels_extract_var_levels()

In [205]:

version_distance = "pearson"
RES, SAVEDIR_MULT, params, list_bregion = load_mult_data_helper(animal, DATE, version_distance)

resthis =[]
resthis = []
for res in RES:
    for bregion in list_bregion:
        for twind in res["list_time_windows"]:
            key = (bregion, twind)
            print("Collecting data for:", key)
            PA = res["DictBregionTwindPA"][key]
            # Clraw = res["DictBregionTwindClraw"][key]
            # Clsim = res["DictBregionTwindClsim"][key]
            EFFECT_VARS = res["EFFECT_VARS"]
    
            # Norm of activity for each level of the variable. Then average these norms over the levels.    
            for var in EFFECT_VARS:
                # condition this on each of the other vars
                for var_other in EFFECT_VARS:
                    if var==var_other:
                        continue
                    
                    # Extract raw data
                    subtract_mean_each_level_of_var = var_other
                    _, _, Clraw, _, PAagg = _preprocess_rsa_scalar_population(PA, [var, var_other], res["version_distance"], 
                                                      subtract_mean_each_level_of_var,
                                                      PLOT = False, sdir=sdir, PLOT_THEORETICAL_SIMMATS=False,
                                                    COMPUTE_SAME_DIFF_DIST = False, COMPUTE_VS_THEOR_MAT = False)
                    
                    # Norms is currently one val for each (var, var_other)
                    norms = np.linalg.norm(Clraw.Xinput, axis=1)

                    # get the mean for each lev of var, averaged over all othervar. 
                    # this is ok ebcuase alrraedy subtracted out effect of each ohervar above.
                    list_lev = Clraw.rsa_labels_extract_var_levels()[var]
                    for lev in list_lev:
                        # get rows with this level
                        inds_ = Clraw.rsa_index_rows_with_this_level(var, lev)
                        norm_this = np.mean(norms[inds_])
                        
                        resthis.append({
                            "bregion":bregion,
                            "twind":twind,
                            "which_level":res["which_level"],
                            "var":var,
                            "var_level":lev,
                            "norm_pop_vec":normthis
                        })    
                                            
                    # # break it into levels of responses
                    # for normthis, lev in zip(norms, Clraw.Labels):
                    #     assert len(lev)==1, "should be 1-tuple"
                    #     resthis.append({
                    #         "bregion":bregion,
                    #         "twind":twind,
                    #         "which_level":res["which_level"],
                    #         "var":var,
                    #         "var_level":lev[0],
                    #         "norm_pop_vec":normthis
                    #     })    

from pythonlib.tools.pandastools import append_col_with_grp_index, plot_subplots_heatmap
dfres_norm_levs = pd.DataFrame(resthis)
dfres_norm_levs
dfres_norm_levs = append_col_with_grp_index(dfres_norm_levs, ["var_level", "which_level", "twind"], "vl_wl_tw", strings_compact=True)
dfres_norm_levs = append_col_with_grp_index(dfres_norm_levs, ["which_level", "twind"], "wl_tw", strings_compact=True)

In [ ]:
# for each level of othervar, slice out just those rows.


In [164]:
list_lev_other = PA.Xlabels["trials"][var_other].unique().tolist()

In [169]:
Clraw.Labels

In [208]:
var = "gap_to_next_angle_binned"
var_other = "stroke_index_semantic"
subtract_mean_each_level_of_var = var_other
_, _, Clraw, _, PAagg = _preprocess_rsa_scalar_population(PA, [var, var_other], res["version_distance"], 
                                  subtract_mean_each_level_of_var,
                                  PLOT = False, sdir=sdir, PLOT_THEORETICAL_SIMMATS=False,
                                COMPUTE_SAME_DIFF_DIST = False, COMPUTE_VS_THEOR_MAT = False)

Clraw.rsa_plot_heatmap(diverge=True)

In [175]:
for lev_other in list_lev_other:
    inds = Clraw.rsa_index_rows_with_this_level(var_other, lev_other)
    print(lev_other, inds)
    levs = Clraw.rsa_index_values_this_var(var, inds)
    
    # 

In [176]:
levs

In [ ]:
# Summarize all. datapt is (bregion, var)



In [154]:
# Each subplot is a bregion... 
# for norm_method in [None, "all_sub", "col_sub", "row_sub"]:
for norm_method in [None]:
    for var in list_var:
        dfthis = dfres_norm_levs[dfres_norm_levs["var"]==var]
        fig, axes = plot_subplots_heatmap(dfthis, "var_level", "wl_tw", "norm_pop_vec", "bregion", share_zlim=False, norm_method=None)
        savefig(fig, f"{savedir}/heatmaps_varlevels-subplot_bregion-var_{var}-norm_{norm_method}.pdf")
        plt.close("all")

In [137]:
if False:
    # Each subplot a variable level. this doesnt work well since big diff across bregions in fr.
    norm_method = "row_sub"
    for var in list_var:
        dfthis = dfres_norm_levs[dfres_norm_levs["var"]==var]
        fig, axes = plot_subplots_heatmap(dfthis, "bregion", "wl_tw", "norm_pop_vec", "var_level", share_zlim=True, norm_method=norm_method)
        plt.close("all")


In [209]:
# Average over levels
### SINGLE SUMMARY:
for norm_method in [None, "all_sub", "col_sub", "row_sub"]:
    fig, axes = plot_subplots_heatmap(dfres_norm_levs, "var", "wl_tw", "norm_pop_vec", "bregion", share_zlim=True, 
                                      norm_method=norm_method)
    savefig(fig, f"{savedir}/heatmaps_meanOverLevs_varlevels-subplot_bregion-norm_{norm_method}.pdf")
    plt.close("all")
    

In [151]:
# for each variable, plot its activity for each level
from pythonlib.tools.listtools import stringify_list
from pythonlib.tools.snstools import rotateLabel
list_var = EFFECT_VARS
# S = 3
# ncols = len(list_var)
# nrows = int(np.ceil(len(list_var)/ncols))
# fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*S, nrows*S), sharex=False, sharey=True)

for var, ax in zip(list_var, axes.flatten()):
    
    dfthis = dfres_norm_levs[dfres_norm_levs["var"]==var]
    fig = sns.catplot(data=dfthis, x="wl_tw", y="norm_pop_vec", col="bregion", hue="var_level", kind="point")
    rotateLabel(fig)
    savefig(fig, f"{savedir}/pointplot-alllevels-{var}.pdf")

    # levs = dfthis["var_level"].tolist()
    # norms = dfthis["norm_pop_vec"].values
    # 
    # levs = stringify_list(levs)
    # 
    # ax.plot(levs, norms, "-o", label=lev)
    # ax.axhline(0, color="k", alpha=0.25)
    # ax.legend()
    # ax.set_title(var)
    # rotate_x_labels(ax, 70)

In [210]:

fig = sns.catplot(data=dfres_norm_levs, x="wl_tw", y="norm_pop_vec", hue="var", col="bregion", kind="point")
rotateLabel(fig)
savefig(fig, f"{savedir}/pointplot-meanOverLevs.pdf")


#### Compute decodability of the variable

In [ ]:
# Relates to how separate are the activity patterns.
